In [14]:
import rasterio
from rasterio.enums import Resampling

with rasterio.open('activities/2015/Agencias de cobranza y comunicaciones.tiff') as src:
    data = src.read(
        out_shape=(src.count, src.height, src.width),
        resampling=Resampling.nearest
    )

    transform = src.transform



In [25]:
import os

# Carpeta base donde buscas subdirectorios con archivos TIFF.
assets_dir = './activities'

# Lista de strings con los nombres de cada subcarpeta.
folders = [f for f in os.listdir(assets_dir) if os.path.isdir(os.path.join(assets_dir, f))]
folders
# Donde el: 
# os.listdir(assets_dir) devuelve todos los nombres (archivos y carpetas)
# El filtro os.path.isdir(...) se queda solo con aquellos que son carpetas.


['2015',
 '2016A',
 '2016B',
 '2017A',
 '2017B',
 '2018A',
 '2018B',
 '2019A',
 '2019B',
 '2020A',
 '2020B',
 '2021A',
 '2021B',
 '2022A',
 '2022B',
 '2023B',
 '2024A',
 '2024B']

In [21]:
first_folder_path = os.path.join(assets_dir, folders[0])  #  Construyes la ruta completa de la primera carpeta
first_folder_files = {f for f in os.listdir(first_folder_path) if f.endswith('.tiff')} # Te todos los archivos raster de la primera carpeta
print(first_folder_path)
print(first_folder_files)

./activities\2015
{'Fabricación de carrocerías, y equipo de transporte etc.tiff', 'Servicios y mantenimiento a automoviles diversos.tiff', 'Tortillerías y panificadoras.tiff', 'Comercio minorista diverso.tiff', 'Fabricación de material de construcción.tiff', 'Fertilizantes y agricultura.tiff', 'Consultorio y otros servicios medicos.tiff', 'Alimentos y bebidas.tiff', 'Cultura esparcimiento y deporte.tiff', 'Hospedaje.tiff', 'Industria del papel.tiff', 'Educacion y otros sdervicios educacion.tiff', 'Manufactura metal y herreria.tiff', 'Generación, transmision, distribucion energía eléctrica.tiff', 'Organismos operadores de agua.tiff', 'Industria alimentaria y de bebidas.tiff', 'Explotacion de animales.tiff', 'Maquiladoras y textileras.tiff', 'Fabricas diversas.tiff', 'Construccion.tiff', 'Alquileres diversos.tiff', 'Petroquimica.tiff', 'Banca y seguros.tiff', 'Servicios de belleza y funerarios.tiff', 'Agencias de cobranza y comunicaciones.tiff', 'Comunicaciones y transportes paqueteria.t

In [22]:

# Get the list of .tiff files in the first folder
first_folder_path = os.path.join(assets_dir, folders[0])
first_folder_files = {f for f in os.listdir(first_folder_path) if f.endswith('.tiff')}

# Contendrá únicamente los nombres de archivo TIFF que aparecen en todas las subcarpetas.
common_tiff_files = first_folder_files.copy()
for folder in folders[1:]: # Iteras sobre el resto de carpetas
    folder_path = os.path.join(assets_dir, folder)
    folder_files = {f for f in os.listdir(folder_path) if f.endswith('.tiff')}
    common_tiff_files.intersection_update(folder_files)


    

In [ ]:
import os

# Create the "activities" folder in the root directory
activities_dir = os.path.join(os.getcwd(), 'imagenes')        # os.getcwd() obtiene la ruta del directorio de trabajo actual.
os.makedirs(activities_dir, exist_ok=True)                    # crea la carpeta imagenes si no existe, sin lanzar error si ya está creada.

# Create a folder for every element in common_tiff_files
for file_name in common_tiff_files:
    folder_name = os.path.splitext(file_name)[0]                # Extrae el nombre sin la extensión .tiff
    folder_path = os.path.join(activities_dir, folder_name)     # Actualizas la direccion
    os.makedirs(folder_path, exist_ok=True)                     # Creas una carpeta con ese nombre dentro de imagenes

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from rasterio.enums import Resampling

colormap = plt.cm.inferno

for tiff_file in common_tiff_files:
    base_name     = os.path.splitext(tiff_file)[0]
    output_folder = os.path.join(activities_dir, base_name)
    # Asegúrate de que existe la carpeta de salida
    os.makedirs(output_folder, exist_ok=True)

    for year_folder in folders:
        src_path = os.path.join(assets_dir, year_folder, tiff_file)
        if not os.path.exists(src_path):
            # Si no existe (aunque en teoría common_tiff_files lo garantiza), lo saltamos
            continue

        print(src_path) # Checar directorio correcto

        # Leemos el TIFF
        with rasterio.open(src_path) as src:
            data = src.read(
                out_shape=(src.count, src.height, src.width),
                resampling=Resampling.nearest
            )
        # Ajustes de tipo y escala
        data = (data * 1000).astype(np.uint16)
        norm = mcolors.Normalize(vmin=data.min(), vmax=data.max())

        # Mapeo de colores
        mapped = colormap(norm(data[0]))
        # Definimos alfa donde haya datos (> mínimo)
        mask = (data[0] > data.min())
        mapped[..., 3] = mask.astype(np.uint8)

        # Guardamos el PNG nombrado por el año/carpeta
        out_png = os.path.join(output_folder, f"{year_folder}.png")
        plt.imsave(out_png, mapped)

./activities\2015\Fabricación de carrocerías, y equipo de transporte etc.tiff


C:\Users\eagel\AppData\Local\Temp\ipykernel_1148\3903006430.py:29: RuntimeWarning: overflow encountered in multiply
  data = (data * 1000).astype(np.uint16)
C:\Users\eagel\AppData\Local\Temp\ipykernel_1148\3903006430.py:29: RuntimeWarning: invalid value encountered in cast
  data = (data * 1000).astype(np.uint16)


./activities\2016A\Fabricación de carrocerías, y equipo de transporte etc.tiff
./activities\2016B\Fabricación de carrocerías, y equipo de transporte etc.tiff
./activities\2017A\Fabricación de carrocerías, y equipo de transporte etc.tiff
./activities\2017B\Fabricación de carrocerías, y equipo de transporte etc.tiff
./activities\2018A\Fabricación de carrocerías, y equipo de transporte etc.tiff
./activities\2018B\Fabricación de carrocerías, y equipo de transporte etc.tiff
./activities\2019A\Fabricación de carrocerías, y equipo de transporte etc.tiff
./activities\2019B\Fabricación de carrocerías, y equipo de transporte etc.tiff
./activities\2020A\Fabricación de carrocerías, y equipo de transporte etc.tiff
./activities\2020B\Fabricación de carrocerías, y equipo de transporte etc.tiff
./activities\2021A\Fabricación de carrocerías, y equipo de transporte etc.tiff
./activities\2021B\Fabricación de carrocerías, y equipo de transporte etc.tiff
./activities\2022A\Fabricación de carrocerías, y equ

Prueba de paletas de colores

In [28]:
import rasterio
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib.colors as mcolors

src_path = 'activities/2015/Agencias de cobranza y comunicaciones.tiff'

colormaps = [
    'viridis', 'plasma', 'inferno', 'magma', 'cividis',
    'Blues', 'Greens', 'Oranges', 'Purples', 'YlGnBu',
    'YlOrRd', 'BuPu', 'PuBuGn', 'BuGn', 'YlOrBr',
    'coolwarm', 'RdBu', 'RdYlBu', 'PRGn', 'BrBG',
]

# Leemos el TIFF solo una vez
with rasterio.open(src_path) as src:
    data = src.read(
        out_shape=(src.count, src.height, src.width),
        resampling=Resampling.nearest
    )
    transform = src.transform

# Ajustes de tipo y escala
data = (data * 1000).astype(np.uint16)
norm = mcolors.Normalize(vmin=data.min(), vmax=data.max())

# Aseguramos que la carpeta de salida exista
os.makedirs("paletas", exist_ok=True)

for color in colormaps:
    colormap = plt.get_cmap(color)
    
    mapped = colormap(norm(data[0]))
    
    # Definimos alfa donde haya datos (> mínimo)
    mask = (data[0] > data.min())
    mapped[..., 3] = mask.astype(np.uint8)

    out_png = os.path.join("paletas", f"{color}.png")
    plt.imsave(out_png, mapped)


C:\Users\eagel\AppData\Local\Temp\ipykernel_1148\3694951293.py:26: RuntimeWarning: overflow encountered in multiply
  data = (data * 1000).astype(np.uint16)
C:\Users\eagel\AppData\Local\Temp\ipykernel_1148\3694951293.py:26: RuntimeWarning: invalid value encountered in cast
  data = (data * 1000).astype(np.uint16)


In [29]:
import rasterio
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib.colors as mcolors

src_path = 'activities/2015/Agencias de cobranza y comunicaciones.tiff'

colormaps = [
    'viridis', 'plasma', 'inferno', 'magma', 'cividis', 'turbo',
    'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
    'YlOrBr', 'YlOrRd', 'YlGn', 'YlGnBu', 'OrRd', 'PuRd',
    'RdPu', 'BuPu', 'PuBu', 'PuBuGn', 'BuGn', 'GnBu',
    'hot', 'afmhot', 'gist_heat', 'bone', 'pink',
    'spring', 'summer', 'autumn', 'winter', 'cool', 'copper'
]

# Leemos el TIFF solo una vez
with rasterio.open(src_path) as src:
    data = src.read(
        out_shape=(src.count, src.height, src.width),
        resampling=Resampling.nearest
    )
    transform = src.transform

# Ajustes de tipo y escala
data = (data * 1000).astype(np.uint16)
norm = mcolors.Normalize(vmin=data.min(), vmax=data.max())

# Aseguramos que la carpeta de salida exista
os.makedirs("paletas", exist_ok=True)

for color in colormaps:
    colormap = plt.get_cmap(color)
    
    mapped = colormap(norm(data[0]))
    
    # Definimos alfa donde haya datos (> mínimo)
    mask = (data[0] > data.min())
    mapped[..., 3] = mask.astype(np.uint8)

    out_png = os.path.join("paletas", f"{color}.png")
    plt.imsave(out_png, mapped)

C:\Users\eagel\AppData\Local\Temp\ipykernel_1148\713500006.py:28: RuntimeWarning: overflow encountered in multiply
  data = (data * 1000).astype(np.uint16)
C:\Users\eagel\AppData\Local\Temp\ipykernel_1148\713500006.py:28: RuntimeWarning: invalid value encountered in cast
  data = (data * 1000).astype(np.uint16)
